In [347]:
import numpy as np
import math
import vtk

In [348]:
from scipy.optimize import fsolve

class EllipsoidIntersection():
    def __init__(self, a = 1., c = 2.):
        self._a = a
        self._c = c
                
        self._ap = a
        self._cp = a
        
        self._u_2 = np.array((0., 1., 0.))
        self._u_1 = np.array((1., 0., 0.))

        self._p_cp = np.array((0., 0., 0.))
        self._c_e = np.array((0., 0., 0.))
                
        self._recompute_intersection = True
        
        
    def _theta_derivative(self, theta, x, u1a, u2c, ex):
        df_theta = (x - u1a*np.sin(theta) - u2c*np.cos(theta) - ex)**2
        return df_theta


    
    def _compute_theta(self, position, u_1, u_2, c_e, ap, cp):
        theta = fsolve(self._theta_derivative, 0.5 * np.pi, args=(position[0], u_2[0]*cp, u_1[0]*ap, c_e[0]), xtol=1.49012e-08)
        print 'Theta found:', theta, 'x:', position[0], 'f_x(theta) = ', self._theta_derivative(theta, position[0], u_2[0]*cp, u_1[0]*ap, c_e[0])
        print 'Theta found:', theta, 'y:', position[1], 'f_y(theta) = ', self._theta_derivative(theta, position[1], u_2[1]*cp, u_1[1]*ap, c_e[1])
        print 'Theta found:', theta, 'z:', position[2], 'f_z(theta) = ', self._theta_derivative(theta, position[2], u_2[2]*cp, u_1[2]*ap, c_e[2])
        return theta

    
    
    def _compute_intersection(self, intersection_plane, d=0.):
        m, n, p = intersection_plane
        
        p_cp = d/(m**2+n**2+p**2) * np.array([m, n, p])
        u_2 = np.array([-p*m, -p*n, m**2+n**2]) / np.sqrt((m**2+n**2)*(m**2+n**2+p**2))
        
        # Compute the scalars that give the intersection of the intersection ellipse with the plane
        ta = (u_2[0]**2+u_2[1]**2)/self._a**2 + u_2[2]**2/self._c**2
        tb = 2.*((u_2[0]*p_cp[0] + u_2[1]*p_cp[1])/self._a**2 + u_2[2]*p_cp[2]/self._c**2)
        tc = (p_cp[0]**2+p_cp[1]**2)/self._a**2 + p_cp[2]**2/self._c**2 - 1.
        
        t_discriminant = tb**2 - 4*ta*tc
        if t_discriminant < 1e-8:
            print 'Warning: there no exists intersection of the ellipsoid with the plane provided'
            self._ap = 0.
            self._cp = 0.
                                
            self._u_2 = np.array((0., 0., 0.))
            self._u_1 = np.array((0., 0., 0.))

            self._p_cp = np.array((0., 0., 0.))
            self._c_e = np.array((0., 0., 0.))

            
        t2_1 = (-tb - np.sqrt(t_discriminant)) / (2*ta)
        t2_2 = (-tb + np.sqrt(t_discriminant)) / (2*ta)
        
        u_t2_1 = t2_1 * u_2 + p_cp
        u_t2_2 = t2_2 * u_2 + p_cp
        c_e = (u_t2_1 + u_t2_2) / 2.
        
        u_1 = np.array([n, -m, 0.]) / np.sqrt(m**2+n**2)

        ta = (u_1[0]**2+u_1[1]**2)/self._a**2 + u_1[2]**2/self._c**2
        tb = 2.*((u_1[0]*c_e[0] + u_1[1]*c_e[1])/self._a**2 + u_1[2]*c_e[2]/self._c**2)
        tc = (c_e[0]**2+c_e[1]**2)/self._a**2 + c_e[2]**2/self._c**2 - 1.
        
        t_discriminant = tb**2 - 4*ta*tc
        
        t1_1 = (-tb - np.sqrt(t_discriminant)) / (2*ta)
        t1_2 = (-tb + np.sqrt(t_discriminant)) / (2*ta)
        
        ap = np.max([t1_1, t1_2])
        cp = np.sqrt(np.dot(u_t2_2 - c_e, u_t2_2 - c_e))
        
        return u_1, u_2, c_e, ap, cp
    
    
    
    def compute_intersection(self, intersection_plane, d=0.):
        self._u_1, self._u_2, self._c_e, self._ap, self._cp = self._compute_intersection(intersection_plane=intersection_plane, d=d)
        
        
        
    def compute_plane_arc(self, theta, previous_position=None):
        theta_0 = 0.
        u_1, u_2, c_e, ap, cp = self._u_1, self._u_2, self._c_e, self._ap, self._cp
        
        if previous_position is not None:
            # Compute the previous intersection and last computed position to generate the current starting angle
            # New samples are taken starting from the computed angle
            theta_0 = self._compute_theta(previous_position, u_1, u_2, c_e, ap, cp)
        
        theta = theta + theta_0
        
        position = c_e[...,np.newaxis] + u_1[...,np.newaxis] * np.cos(theta)[np.newaxis] * ap + u_2[...,np.newaxis] * np.sin(theta)[np.newaxis] * cp
        return position.T


In [349]:
def draw_vector(src, dst, renderer, color='Banana'):
    lineSource = vtk.vtkLineSource()
    lineSource.SetPoint1(src)
    lineSource.SetPoint2(src + dst)

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(lineSource.GetOutputPort())
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetLineWidth(1.0)
    actor.GetProperty().SetColor(colors.GetColor3d(color))

    renderer.AddActor(actor)
    
def draw_sphere(center, renderer, radii=0.05, color='Tomato'):
    # create a Sphere
    sphereSource = vtk.vtkSphereSource()
    sphereSource.SetCenter(center)
    sphereSource.SetRadius(radii)
    
    # create a mapper
    sphereMapper = vtk.vtkPolyDataMapper()
    sphereMapper.SetInputConnection(sphereSource.GetOutputPort())

    # create an actor
    sphereActor = vtk.vtkActor()
    sphereActor.SetMapper(sphereMapper)
    sphereActor.GetProperty().SetColor(colors.GetColor3d(color))

    # add the actors to the scene
    renderer.AddActor(sphereActor)

In [350]:
theta = np.linspace(0., 2.*math.pi, 20)
phi = np.linspace(-math.pi/2., math.pi/2., 20)

theta, phi = np.meshgrid(theta, phi)
theta = theta.flatten()
phi = phi.flatten()

u = np.linspace(-1., 1., 20)
v = np.linspace(-1., 1., 20)
u, v = np.meshgrid(u, v)
u = u.flatten()
v = v.flatten()

# Ellipsoid parameters:
a = 1.
c = 2.

# Plane parameters
m = 1.
n = 0.
o = 1e-1
p = o
d = 0.0


In [351]:
# Compute ellipsoid
xe = a*np.cos(theta)*np.cos(phi)
ye = a*np.sin(theta)*np.cos(phi)
ze = c*np.sin(phi)

# Compute plane
xp = u
yp = v
zp = (d - m*xp - n*yp)/o

p_ori = np.array([0., 0., 0.])
p_0 = p_ori
p_c = np.array([0., 0., d/o])
p_cp = d/(m**2+n**2+p**2) * np.array([m, n, o])

In [352]:
ellipsoid_intersection = EllipsoidIntersection(a = a, c = c)
ellipsoid_intersection.compute_intersection((m, n, p), d)

points = ellipsoid_intersection.compute_plane_arc(np.linspace(0.0, 0.25, 10) * np.pi)
points = np.vstack((points, ellipsoid_intersection.compute_plane_arc(np.linspace(0.3, 1.25, 10) * np.pi)))
(m1, n1, p1) = np.cross(points[-1], (m, n, p))
d1 = np.dot(points[-1], (m1, n1, p1))

ellipsoid_intersection.compute_intersection((m1, n1, p1), d1)
points2 = ellipsoid_intersection.compute_plane_arc(np.linspace(0.0, 1., 10) * np.pi, points[-1])
print 'm1:', m1, 'n1:', n1, 'p1:', p1, 'd1:', d1

Theta found: [1.6741755] x: 0.1386750490563073 f_x(theta) =  [2.78104284e-31]
Theta found: [1.6741755] y: 0.7071067811865477 f_y(theta) =  [1.98043737]
Theta found: [1.6741755] z: -1.3867504905630728 f_z(theta) =  [7.77016987]
m1: 0.07071067811865477 n1: -1.4006179954687035 p1: -0.7071067811865477 d1: 0.0


In [353]:
colors = vtk.vtkNamedColors()
renderer = vtk.vtkRenderer()

In [354]:
print('Rendering ellipsoid')
pointsEllipsoid = vtk.vtkPoints()

theta = np.linspace(0., 2.*math.pi, 20)
phi = np.linspace(-math.pi/2., math.pi/2., 20)
i = 0
for p in phi:
    for t in theta:
        x_i = a*np.cos(t)*np.cos(p)
        y_i = a*np.sin(t)*np.cos(p)
        z_i = c*np.sin(p)
        pointsEllipsoid.InsertPoint(i, (x_i, y_i, z_i))
        i+=1

ugridEllipsoid = vtk.vtkUnstructuredGrid()
ugridEllipsoid.Allocate(400)

for i in range(phi.size-1):
    for j in range(theta.size-1):
        """
        curr_triangle = [i*theta.size+j, i*theta.size+j+1,(i+1)*theta.size+j+1]
        ugridEllipsoid.InsertNextCell(vtk.VTK_TRIANGLE, 3, curr_triangle)
        curr_triangle = [i*theta.size+j, (i+1)*theta.size+j+1,(i+1)*theta.size+j]
        ugridEllipsoid.InsertNextCell(vtk.VTK_TRIANGLE, 3, curr_triangle)
        """
        curr_side = [i*theta.size+j, i*theta.size+j+1]
        ugridEllipsoid.InsertNextCell(vtk.VTK_LINE, 2, curr_side)
        curr_side = [i*theta.size+j+1,(i+1)*theta.size+j+1]
        ugridEllipsoid.InsertNextCell(vtk.VTK_LINE, 2, curr_side)
        curr_side = [(i+1)*theta.size+j+1, i*theta.size+j]
        ugridEllipsoid.InsertNextCell(vtk.VTK_LINE, 2, curr_side)
        
        
ugridEllipsoid.SetPoints(pointsEllipsoid)

ugridMapperEllipsoid = vtk.vtkDataSetMapper()
ugridMapperEllipsoid.SetInputData(ugridEllipsoid)

ugridActorEllipsoid = vtk.vtkActor()
ugridActorEllipsoid.SetMapper(ugridMapperEllipsoid)
ugridActorEllipsoid.GetProperty().SetColor(colors.GetColor3d("Peacock"))
ugridActorEllipsoid.GetProperty().SetEdgeColor(colors.GetColor3d("White"))
ugridActorEllipsoid.GetProperty().EdgeVisibilityOn()

renderer.AddActor(ugridActorEllipsoid)

Rendering ellipsoid


In [355]:
print('Rendering plane')
pointsPlane = vtk.vtkPoints()

xp_i = -2.
yp_i = -2.
zp_i = (d - m*xp_i - n*yp_i) / o
pointsPlane.InsertPoint(0, (xp_i, yp_i, zp_i))
xp_i =  2.
yp_i = -2.
zp_i = (d - m*xp_i - n*yp_i) / o
pointsPlane.InsertPoint(1, (xp_i, yp_i, zp_i))
xp_i =  2.
yp_i =  2.
zp_i = (d - m*xp_i - n*yp_i) / o
pointsPlane.InsertPoint(2, (xp_i, yp_i, zp_i))
xp_i = -2.
yp_i =  2.
zp_i = (d - m*xp_i - n*yp_i) / o
pointsPlane.InsertPoint(3, (xp_i, yp_i, zp_i))

ugridPlane = vtk.vtkUnstructuredGrid()
ugridPlane.Allocate(4)
ugridPlane.InsertNextCell(vtk.VTK_QUAD, 4, [0, 1, 2, 3])
ugridPlane.SetPoints(pointsPlane)

ugridMapperPlane = vtk.vtkDataSetMapper()
ugridMapperPlane.SetInputData(ugridPlane)

ugridActorPlane = vtk.vtkActor()
ugridActorPlane.SetMapper(ugridMapperPlane)
ugridActorPlane.GetProperty().SetColor(colors.GetColor3d("Tomato"))
ugridActorPlane.GetProperty().EdgeVisibilityOn()

renderer.AddActor(ugridActorPlane)

Rendering plane


In [356]:
print('Rendering plane')
pointsPlane = vtk.vtkPoints()

xp_i = -2.
yp_i = -2.
zp_i = (d1 - m1*xp_i - n1*yp_i) / p1
pointsPlane.InsertPoint(0, (xp_i, yp_i, zp_i))
xp_i =  2.
yp_i = -2.
zp_i = (d1 - m1*xp_i - n1*yp_i) / p1
pointsPlane.InsertPoint(1, (xp_i, yp_i, zp_i))
xp_i =  2.
yp_i =  2.
zp_i = (d1 - m1*xp_i - n1*yp_i) / p1
pointsPlane.InsertPoint(2, (xp_i, yp_i, zp_i))
xp_i = -2.
yp_i =  2.
zp_i = (d1 - m1*xp_i - n1*yp_i) / p1
pointsPlane.InsertPoint(3, (xp_i, yp_i, zp_i))

ugridPlane = vtk.vtkUnstructuredGrid()
ugridPlane.Allocate(4)
ugridPlane.InsertNextCell(vtk.VTK_QUAD, 4, [0, 1, 2, 3])
ugridPlane.SetPoints(pointsPlane)

ugridMapperPlane = vtk.vtkDataSetMapper()
ugridMapperPlane.SetInputData(ugridPlane)

ugridActorPlane = vtk.vtkActor()
ugridActorPlane.SetMapper(ugridMapperPlane)
ugridActorPlane.GetProperty().SetColor(colors.GetColor3d("Purple"))
ugridActorPlane.GetProperty().EdgeVisibilityOn()

renderer.AddActor(ugridActorPlane)

Rendering plane


In [357]:
print('Rendering intersection ellipe')
#pointsIntersection = vtk.vtkPoints()

for t_i, curr_position in enumerate(points):
    print(curr_position)
    #pointsIntersection.InsertPoint(t_i, curr_position)
    draw_sphere(curr_position, renderer, radii=0.05, color='Blue')

for t_i, curr_position in enumerate(points2):
    print(curr_position)
    #pointsIntersection.InsertPoint(t_i, curr_position)
    draw_sphere(curr_position, renderer, radii=0.05, color='Banana')
    
"""
ugridIntersection = vtk.vtkUnstructuredGrid()
ugridIntersection.Allocate(20)
for i in range(points.size-1):
    curr_line = [i, i+1]
    ugridIntersection.InsertNextCell(vtk.VTK_LINE, 2, curr_line)
    
ugridIntersection.SetPoints(pointsIntersection)

ugridMapperIntersection = vtk.vtkDataSetMapper()
ugridMapperIntersection.SetInputData(ugridIntersection)

ugridActorIntersection = vtk.vtkActor()
ugridActorIntersection.SetMapper(ugridMapperIntersection)
ugridActorIntersection.GetProperty().SetColor(colors.GetColor3d("Blue"))
ugridActorIntersection.GetProperty().EdgeVisibilityOn()

renderer.AddActor(ugridActorIntersection)
"""

Rendering intersection ellipe
[ 0. -1.  0.]
[-0.01709265 -0.9961947   0.17092647]
[-0.03405521 -0.98480775  0.34055209]
[-0.05075859 -0.96592583  0.50758591]
[-0.06707567 -0.93969262  0.67075669]
[-0.08288226 -0.90630779  0.8288226 ]
[-0.09805807 -0.8660254   0.98058068]
[-0.11248759 -0.81915204  1.12487594]
[-0.12606102 -0.76604444  1.26061022]
[-0.13867505 -0.70710678  1.38675049]
[-0.15866129 -0.58778525  1.58661286]
[-0.18754679 -0.2923717   1.87546793]
[-0.19599667  0.0348995   1.95996666]
[-0.18309019  0.35836795  1.83090185]
[-0.15023368  0.64278761  1.50233676]
[-0.10100728  0.8571673   1.01007277]
[-0.04077484  0.9781476   0.40774837]
[ 0.02390054  0.99254615 -0.23900545]
[ 0.08597166  0.89879405 -0.85971655]
[ 0.13867505  0.70710678 -1.38675049]
[ 0.13867505 -0.70017335  1.40075195]
[ 0.46880972 -0.615765    1.26657163]
[ 0.74239901 -0.45708629  0.97962407]
[ 0.92644403 -0.24327624  0.57451939]
[ 9.98746225e-01 -1.23476023e-04  1.00119200e-01]
[ 0.95058489  0.24304418 -0.3863

'\nugridIntersection = vtk.vtkUnstructuredGrid()\nugridIntersection.Allocate(20)\nfor i in range(points.size-1):\n    curr_line = [i, i+1]\n    ugridIntersection.InsertNextCell(vtk.VTK_LINE, 2, curr_line)\n    \nugridIntersection.SetPoints(pointsIntersection)\n\nugridMapperIntersection = vtk.vtkDataSetMapper()\nugridMapperIntersection.SetInputData(ugridIntersection)\n\nugridActorIntersection = vtk.vtkActor()\nugridActorIntersection.SetMapper(ugridMapperIntersection)\nugridActorIntersection.GetProperty().SetColor(colors.GetColor3d("Blue"))\nugridActorIntersection.GetProperty().EdgeVisibilityOn()\n\nrenderer.AddActor(ugridActorIntersection)\n'

In [358]:
renderWindow = vtk.vtkRenderWindow()
renderWindow.SetWindowName("3D ellipse")
renderWindow.AddRenderer(renderer)

# an interactor
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

renderer.ResetCamera()
renderWindow.Render()

# begin mouse interaction
renderWindowInteractor.Start()